# [Module 1.1] 데모 환경 셋업 및 엣지 디바이스(EC2) 생성 

이 노트북은 아래와 같은 주요 작업을 합니다.
### 1. Demo Setup
- (1) 세이지 메이커 노트북이 사용하는 Role에 아래 정책 추가(예:EdgeManager-Demo-Notebook-gonsoo )
    - AmazonEC2FullAccess
    - AmazonEC2RoleforSSM
    - AmazonSSMManagedInstanceCore
    - AmazonSSMFullAccess
    - AWSIoTFullAccess
- 위의 정책들은 세이지메이커 노트북이 위 Role을 통해서 EC2, SSM, IoT 서비스를 이용 함.


#### 2. Launch EC2 인스턴스

- (1) EC2에 사용할 key pair 생성
- (2) EC2 사용할 Role 생성 (예: EdgeManager-Demo-EC2-gonsoo) 및 아래 정책 추가
    - 아래의 정책이 모두 추가 되어야 함
        * AmazonS3FullAccess 
        * AmazonSSMManagedInstanceCore 
        * CloudWatchAgentAdminPolicy

- (3) 세이지메이커 노트북에서 사용하는 역할(예: EdgeManager-Demo-Notebook-gonsoo) 에 아래의 정책을 추가
```
    {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "iam:PassRole",
                "Resource": "arn:aws:iam::<account>:role/<role-name>" # EC2 생성 Role 임
            }
        ]
    }
```
- (4) IAM 프로파일(예: edgemanager2client2gsmoon) 를 가지고 EC2 인스턴스 생성
- (5) EC2에 SSH 로 접근 테스트
    * ssh key 를 다운로드
    * sudo chmod 400 iot-sendbox-ec2-key-pair.pem
    * ssh -i "iot-sendbox-ec2-key-pair.pem" ubuntu@ec2-XXXXX6.us-east-2.compute.amazonaws.com



## Introduction

세이지메이커 에지 관리자는 세이지메이커의 서비스로서 아래와 같은 작업을 할 수 있습니다.

+ 엣지 장치 하드웨어용 맞춤형 모델 준비합니다
+ 엣지 장치에서 기계 학습 추론을 효율적으로 실행하기 위한 런타임을 포함합니다.
+ 엣지 장치에서 각 모델의 데이터 샘플을 안전하게 SageMaker로 전송하여 라벨 재지정 및 재훈련을 수행할 수 있습니다.

이 서비스에는 두 가지 주요 구성 요소가 있습니다.
+ 클라우드에서의 세이지메이커 엣지 매니저 
+ 엣지 장치의 세이지메이커 엣지 에이전트

이 전체 Nootebook은 에지 장치에서 실행중인 Sagemaker Edge를 가져 오는 엔드 투 엔드 워크플로를 보여줍니다.여기에는 다음 단계가 포함됩니다.

+ 세이지메이커 네오를 사용하여 모델을 컴파일
+ 세이지메이커 에지 관리자로 컴파일 된 모델 패키지
+ 세이지메이커 에지 관리자 에이전트를 사용하여 배포
+ 모델과 함께 추론 실행
+ 모델의 입력 및 출력 데이터를 S3로 캡처
+ 엣지에서의 실형 결과를 CloudWatch로 전송
+ 엣지 추가의 예제 (역자가 기존 샘플에 없는 것을 추가 함)

**참고**:
일반적으로 SageMaker 에지 에이전트는 에지 장치에서 실행됩니다.이 노트북을 위해 EC2 인스턴스에서 에이전트를 실행합니다.컴파일된 모델을 패키지화한 다음 Edge 장치의 에이전트에 로드하여 예측을 수행하는 방법을 보여 줍니다.마지막으로 Agent를 통해 모델의 입력 및 출력을 S3로 캡처하는 방법을 보여 줍니다.

이 노트북은 노트북 인스턴스에만 사용됩니다.이 노트북을 실행할 때 커널을 선택하십시오: `conda_tensorflow_p36`

**참고**: 이 노트북의 사용에는 가격 영향이 있습니다.자세한 내용은 [엣지 매니저](https://aws.amazon.com/sagemaker/edge-manager/pricing) 를 참조하십시오.

## Demo Setup

SageMaker 액세스 권한이 있는 AWS 계정 역할이 필요합니다.이 역할은 SageMaker에 S3에 대한 액세스 권한을 부여하고, EC2 인스턴스를 시작하고, 시스템 관리자를 사용하여 명령을 전송하는 데 사용됩니다.

In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [2]:
print(role)

arn:aws:iam::057716757052:role/EdgeManager-Demo-Notebook-gonsoo


**[중요] [IAM 콘솔](https://console.aws.amazon.com/iam) 에서 위에 프린트된 세이지메이커 역할을 찾은 후 다음 정책을 찾아 역할에 연결합니다.**

- AmazonEC2FullAccess 
- AmazonEC2RoleforSSM 
- AmazonSSMManagedInstanceCore 
- AmazonSSMFullAccess 
- AWSIoTFullAccess 


역할에 정책을 연결하는 방법에 대한 자세한 내용은 [여기](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-attach-detach.html#add-policies-console) 을 참조하십시오.

[참고] 실제 디바이스에서 이 예제를 시도할 경우 AWSIOTFullAccess만 연결하여 AWS IoT에서 인증서를 생성합니다.


#### S3 버킷 준비
컴파일 후 생성 된 모델 아티팩트와 에지 패키징 작업 후에 생성 된 패키지 아티팩트를 저장하는 데 사용되는 S3 버킷이 필요합니다.

In [3]:
# S3 bucket and folders for saving model artifacts.
# bucket = <Bucket Name> 지정된 버켓이름을 기술해도 됩니다.
bucket = sess.default_bucket()  # 계정의 디폴트 버켓 이름 가져오기
folder = 'DEMO-Sandbox-Sagemaker-Edge'
iot_folder = folder + '/iot'



테스트 이미지를 S3 버킷에 업로드합니다.이 이미지는 나중에 추론에 사용됩니다.

In [7]:
keras_img_path = sess.upload_data('keras.bmp', bucket, iot_folder)
print("keras_img_path: ", keras_img_path)

keras_img_path:  s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/iot/keras.bmp


### Launch EC2 Instance

앞서 언급했듯이 이 EC2 인스턴스는 에이전트 소프트웨어를 실행하기 위해 Edge 디바이스 대신 사용됩니다.

In [10]:
ec2_client = boto3.client('ec2', region_name=region)

EC2 인스턴스 용 키 페어를 생성하고 키 pem 파일을 저장하십시오.이 키를 SSH와 함께 사용하여 인스턴스에 연결할 수 있습니다. 
- **[중요] 로컬 랩탑에 키 pem 파일을 다운로드 하여 저장하세요.** 이후에 SSH로 접근할 예정 입니다.
    - 추후에 아래와 같은 명령으로 SSH로 EC2에 접근 합니다.
        - ```ssh -i "iot-sendbox-ec2-key-pair.pem" ubuntu@ec2XXXX-XXX.us-east-2.compute.amazonaws.com```
- 또한 AWS Systems Manager를 사용하여 인스턴스에 명령을 보냅 예정 입니다.

In [15]:
key_pairs = ec2_client.describe_key_pairs()
key_names = list(map(lambda x : x['KeyName'], key_pairs['KeyPairs']))

key_name = 'iot-sendbox-ec2-key-pair' # 원하는 이름으로 바꾸세요.

if key_name in key_names:
    ec2_key_pair = ec2_client.delete_key_pair(
        KeyName=key_name,
    )

In [16]:
ec2_key_pair = ec2_client.create_key_pair(
    KeyName=key_name,
)

key_pair = str(ec2_key_pair['KeyMaterial'])
key_pair_file = open('iot-sendbox-ec2-key-pair.pem','w')
key_pair_file.write(key_pair)
key_pair_file.close()

#### (1) EC2 사용할 Role 생성 (예: EdgeManager-Demo-EC2-gonsoo)
사용할 EC2 인스턴스에 대한 역할을 만듭니다.[Amazon EC2를 위한 IAM 역할](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html) 에 대한 자세한 내용을 참조하십시오.

**다음 단계를 수행하여 [IAM 역할 만들기](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/iam-roles-for-amazon-ec2.html#create-iam-role)역할 이름과 역할 ARN을 기록합니다.**

#### (2) 생성 후 다음 정책이 역할에 연결되어 있는지 확인합니다.

- AmazonS3FullAccess 
- AmazonSSMManagedInstanceCore 
- CloudWatchAgentAdminPolicy 

이후에 역할 이름은 EC2 인스턴스를 시작할 때 사용되며, 인라인 정책을 생성하려면 역할 ARN이 필요합니다.

#### (3) 노트북에서 사용하는 역할(예: EdgeManager-Demo-Notebook-gonsoo) 에 아래의 정책을 추가
[IAM 콘솔](https://console.aws.amazon.com/iam) 에서 이 노트북에 대해 사용하는 동일한 세이지메이커 역할을 찾은 다음 역할 요약 페이지에서 '인라인 정책 추가' 버튼을 클릭하고 JSON 형식을 선택한 다음 내용을 아래 문으로 바꿉니다.

다음 콘텐츠를 복사하기 전에 `IAM:passrole` 작업에 대해 '리소스' 필드에서 방금 만든 EC2 역할 ARN을 사용해야 합니다.
* EdgeManager-Demo-Notebook-gonsoo 역할이 EC2에 사용하는 역할 기반위에서 iam:PassRole을 수행할 수 있게 함.


```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::<account>:role/<role-name>"
        }
    ]
}
예제:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": "arn:aws:iam::XXXXXXX:role/EdgeManager-Demo-EC2-gonsoo"
        }
    ]
}


```

## EC2 인스턴스 생성

EC2 C5 인스턴스를 시작합니다.이 예에서 우리는 aws 딥 러닝 AMI를 사용합니다.

In [17]:
ami_map = {
    'us-east-1': 'ami-063585f0e06d22308',
    'us-east-2': 'ami-01bd6a1621a6968d7',
    'us-west-2': 'ami-0bc87a16c757a7f07',
    'eu-central-1': 'ami-01227276a4e5a4a31',
    'ap-northeast-1': 'ami-03b8cfea5460e4881',
    'eu-west-1': 'ami-006ff58f5247c50eb'
}

In [19]:
# the name of the role created for EC2
ec2_profile_name = "EdgeManager-Demo-EC2-gonsoo"  # 위에서 EC2를 위해 생성한 Role 이름을 넣으세요

ec2_instance = ec2_client.run_instances(
     ImageId=ami_map[region],
     MinCount=1,
     MaxCount=1,
     InstanceType='c5.large',
     KeyName= key_name,
     IamInstanceProfile={
        'Name': ec2_profile_name}
)

In [21]:
instance_id = ec2_instance['Instances'][0]['InstanceId'] # will used for running inference later
print("EC2 Instance ID: ", instance_id)

EC2 Instance ID:  i-0aabb20c139dde1af


다음 노트북에서 사용할 변수를 저장 합니다.

In [27]:
%store key_name
%store ec2_profile_name
%store instance_id
%store keras_img_path
%store bucket

Stored 'key_name' (str)
Stored 'ec2_profile_name' (str)
Stored 'instance_id' (str)
Stored 'keras_img_path' (str)
Stored 'bucket' (str)
